In [1]:
import os

os.environ["HF_HOME"] = "/projects/bhuang/.cache/huggingface"
# os.environ["OMP_NUM_THREADS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from datasets import load_dataset, DatasetDict

/home/bhuang/anaconda3/envs/ocd-10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
input_files = {
    # quaero medline
    "quaero_medline_train": "/home/bhuang/icd_10/data/quaero_icd10_by_category/drbenchmark_quaero-medline-train-cls-mistral_large_instruct_2407-processed.jsonl",
    "quaero_medline_valid": "/home/bhuang/icd_10/data/quaero_icd10_by_category/drbenchmark_quaero-medline-validation-cls-mistral_large_instruct_2407-processed.jsonl",
    "quaero_medline_test": "/home/bhuang/icd_10/data/quaero_icd10_by_category/drbenchmark_quaero-medline-test-cls-mistral_large_instruct_2407-processed.jsonl",
    # quaero emea
    "quaero_emea_train": "/home/bhuang/icd_10/data/quaero_icd10_by_category/drbenchmark_quaero-emea-train-cls-mistral_large_instruct_2407-processed.jsonl",
    "quaero_emea_valid": "/home/bhuang/icd_10/data/quaero_icd10_by_category/drbenchmark_quaero-emea-validation-cls-mistral_large_instruct_2407-processed.jsonl",
    "quaero_emea_test": "/home/bhuang/icd_10/data/quaero_icd10_by_category/drbenchmark_quaero-emea-test-cls-mistral_large_instruct_2407-processed.jsonl",
}

In [17]:
input_files_ = [v for k, v in input_files.items() if "medline" in k]
# input_files_ = [v for k, v in input_files.items() if "emea" in k]

dataset = load_dataset("json", data_files=input_files_, split="train")
dataset

Dataset({
    features: ['text', 'has_diso', 'labels'],
    num_rows: 2374
})

In [18]:
# keep only text w/ labels
dataset = dataset.filter(lambda x: x["labels"], num_proc=8)
dataset

Dataset({
    features: ['text', 'has_diso', 'labels'],
    num_rows: 1867
})

In [19]:
train_testvalid = dataset.train_test_split(test_size=0.2)  # , stratify_by_column="labels")
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
new_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']
})
new_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'has_diso', 'labels'],
        num_rows: 1493
    })
    validation: Dataset({
        features: ['text', 'has_diso', 'labels'],
        num_rows: 187
    })
    test: Dataset({
        features: ['text', 'has_diso', 'labels'],
        num_rows: 187
    })
})

In [20]:
for name, ds in new_dataset.items():
    output_file = f"/home/bhuang/icd_10/data/quaero_icd10_by_category_resplitted_new/drbenchmark_quaero-medline-{name}-cls-mistral_large_instruct_2407-processed.jsonl"
    # output_file = f"/home/bhuang/icd_10/data/quaero_icd10_by_category_resplitted_new/drbenchmark_quaero-emea-{name}-cls-mistral_large_instruct_2407-processed.jsonl"
    
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    ds.to_json(output_file, orient="records", lines=True, force_ascii=False)

Creating json from Arrow format: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 364.09ba/s]
